In [2]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames

In [3]:
df = DataFrame(CSV.File("final_data.csv"));

In [4]:
names(df)

38-element Vector{String}:
 "Datetime"
 "Day"
 "Month"
 "Hour"
 "Year"
 "Dual Fuel_Gen_MW"
 "Hydro_Gen_MW"
 "Natural Gas_Gen_MW"
 "Nuclear_Gen_MW"
 "Other Fossil Fuels_Gen_MW"
 "Solar_Gen_MW"
 "Wind_Gen_MW"
 "temp"
 ⋮
 "uvindex"
 "conditions_Clear"
 "conditions_Overcast"
 "conditions_Partly_cloudy"
 "conditions_Rain_overcast"
 "conditions_Rain_partly_cloudy"
 "conditions_Snow_overcast"
 "conditions_Snow_rain_overcast"
 "preciptype_no_precipitation"
 "preciptype_rain"
 "preciptype_rain_and_snow"
 "preciptype_snow"

In [27]:
describe(df)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,Datetime,,2021-01-01 00:00:00,,2021-12-31 23:00:00
2,Day,15.7207,1,16.0,31
3,Month,6.52643,1,7.0,12
4,Hour,11.5011,0,12.0,23
5,Year,2021.0,2021,2021.0,2021
6,Dual Fuel_Gen_MW,45271.4,11226.0,41720.0,170491.0
7,Hydro_Gen_MW,38502.6,13771.0,39276.0,72251.0
8,Natural Gas_Gen_MW,38654.3,12595.0,37104.0,100129.0
9,Nuclear_Gen_MW,43465.0,28202.0,40014.0,89400.0


In [5]:
data = CSV.File("final_data.csv", header=true) |> Tables.matrix;

In [6]:
#Part d
function stable_regression(X, y, k)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    n, p = size(X)
    
    @variable(m, B[1:p])
    @variable(m, u[1:n])
    @variable(m, θ)
    #println(typeof(B))
    #println(typeof(u))
    #println(typeof(y))
    #println(typeof(X))
    @constraint(m, u .+ θ .>= (y - X*B).^2)
    @constraint(m, u .>= 0)
    @objective(m, Min, k*θ + sum(u))

    optimize!(m)
    println(value.(u))
    return value.(B)
end;

In [7]:
y_solar = data[:,11];
X = data[:,13:38];
y_wind = data[:,12];


In [45]:
X

8759×26 Matrix{Any}:
 32.4  29.7  24.0  70.72  0.0  0  0.0   …  0  0  0  1  0  0  0  0  1  0  0  0
 32.5  32.5  24.1  70.76  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.6  28.3  24.2  73.84  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.6  26.8  25.1  76.66  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 31.8  27.2  25.2  76.31  0.0  0  0.0      0  0  1  0  0  0  0  0  1  0  0  0
 29.5  23.6  24.1  79.99  0.0  0  0.0   …  0  0  0  1  0  0  0  0  1  0  0  0
 29.7  23.9  23.2  76.63  0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.9  22.9  21.3  72.8   0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.1  24.7  21.0  74.55  0.0  0  0.0      0  0  0  1  0  0  0  0  1  0  0  0
 28.8  22.7  21.2  72.71  0.0  0  2.35     1  0  0  1  0  0  0  0  1  0  0  0
 30.7  23.1  22.1  69.99  0.0  0  0.0   …  1  0  0  1  0  0  0  0  1  0  0  0
 31.9  24.0  22.1  66.79  0.0  0  0.0      1  0  1  0  0  0  0  0  1  0  0  0
 33.0  23.8  21.3  61.67  0.0  0  0.0      

In [9]:
function MSE(y, pred) 
    return sum((y-pred).^2)/length(y)
end;

### SOLAR: BEST SPLIT
80% TRAIN 20% VALIDATION

In [10]:
n,p = size(X)
MSE_best, k_best = 0, 0
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_solar[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_solar[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp >= MSE_best
        MSE_best = MSE_temp
        k_best = k 
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
[3.0547565462581547e13, 3.054756528639525e13, 3.0547565302352133e13, 3.0547565299109984e13, 3.0547565301374895e13, 3.054756530581398e13, 3.0547565308589203e13, 3.0547565303879215e13, 3.0547565323548094e13, 3.054756534309459e13, 3.0547566068203477e13, 3.054756596726169e13, 3.0547565311089535e13, 3.0547565312879465e13, 3.0547565320946996e13, 3.0547565770990906e13, 3.0547565299252633e13, 3.0547565291519688e13, 3.0547565291309184e13, 3.054756529127077e13, 3.0547565295957785e13, 3.054756530072352e13, 3.054756577838772e13, 3.0547565292521613e13, 3.0547565302183246e13, 3.0547565309709734e13, 3.0547565333002242e13, 3.0547565348378816e13, 3.0547565496604605e13, 3.0547565416166832e13, 3.054756544084406e13, 3.0547565349462883e13, 3.054756533064016e13, 3.0547565379001777e13, 3.0547565288500754e13, 3.0547565270102273e13, 3.054756531774122e13, 3.0547565469133758e13, 3.0547565632953516e13, 3.054756537471729e13,

Excessive output truncated after 524308 bytes.

, 3.3226791148852477e9, 3.322777323279643e9, 3.322699099699752e9, 3.322681076018349e9, 3.3228888862384744e9, 3.3227160458362865e9, 3.322828849826662e9, 3.3228703825740013e9, 3.3228128589768e9, 3.3228221187136316e9, 3.3228059634668603e9, 3.3228021641542563e9, 3.322798359409582e9, 3.3227793279461e9, 3.3227890859579153e9, 3.322764645912661e9, 3.3227680453778915e9, 3.3227382853246937e9, 3.322730616637343e9, 3.3227857903472056e9, 3.322744450297818e9, 3.322722856869894e9, 3.322703128824308e9, 3.3226880649876604e9, 3.322681198951777e9, 3.3227267630480866e9, 3.322712816920147e9, 3.322754230061298e9, 3.3227646097799845e9, 3.3227991923987184e9, 3.3227537245303397e9, 3.3228288812983427e9, 3.322830263216772e9, 3.3228284532123146e9, 3.32280822834152e9, 3.3228375180203314e9, 3.322818267394226e9, 3.3228239387843866e9, 3.322760662479671e9, 3.3227670843478975e9, 3.322764507244221e9, 3.322680891504763e9, 3.322678558821842e9, 3.32267557956265e9, 3.3226785865685945e9, 3.3226757357950945e9, 3.3226754036831

In [11]:
k_best #best split is 60% training and 40% validation

5255

In [12]:
MSE_best

93793.23949012169

In [13]:
X_train_solar = X_train_best
X_valid_solar = X_valid_best
y_train_solar = y_train_best
y_valid_solar = y_valid_best;

In [14]:
#transform back to dataframe to understand the coefficients
X_train_solar_df=DataFrame(X_train_solar, :auto)
X_valid_solar_df=DataFrame(X_valid_solar, :auto)
#rename columns with list of values
rename!(X_train_solar_df, names(df[:, 13:38]))
rename!(X_valid_solar_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_solar=DataFrame(Solar_Gen_MW = y_train_solar)
y_valid_solar=DataFrame(Solar_Gen_MW = y_valid_solar)
#save as CSV
CSV.write("X_train_solar.csv", X_train_solar_df)
CSV.write("X_valid_solar.csv", X_valid_solar_df)
CSV.write("y_train_solar.csv", y_train_solar)
CSV.write("y_valid_solar.csv", y_valid_solar);

### WIND: BEST SPLIT 
90% TRAIN 10% VALIDATION

In [15]:
n,p = size(X)
MSE_best, k_best = 0, 0
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_wind[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_wind[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp >= MSE_best
        MSE_best = MSE_temp
        k_best = k 
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
[3.343147080158134e8, 3.3004617753604025e8, 3.3091247940874654e8, 3.324246222637094e8, 3.306752811085572e8, 3.31972530474928e8, 3.3013543982345587e8, 3.3017130340040374e8, 3.3503483310491586e8, 3.3017208893697923e8, 3.301170733310933e8, 3.299778479969286e8, 3.4084842541220105e8, 3.3465845750374234e8, 3.3167163678375226e8, 3.299694035347467e8, 3.3104175085329735e8, 3.32035292213334e8, 3.33896061188902e8, 3.303958090450038e8, 3.3967358052865684e8, 3.335099447971005e8, 3.376405396600132e8, 3.3797346594418347e8, 3.3281656467330074e8, 3.3349470241435885e8, 3.312376379590643e8, 3.300111625404151e8, 3.3391519152627826e8, 3.304723645067613e8, 3.3250427506490076e8, 3.3549326265056235e8, 3.306928990696955e8, 3.3025773541848516e8, 3.4228299166038716e8, 3.705658832580117e8, 3.4757869885586524e8, 3.4507751819610256e8, 3.553523457560525e8, 3.3043424738680035e8, 3.3488586747486925e8, 3.399474244927652e8, 3.3244

Excessive output truncated after 524296 bytes.

, 1.7857347621641523e8, 1.7272665662322384e8, 1.7020285407142267e8, 1.6964694149239534e8, 1.6937156403864062e8, 1.7137278185737738e8, 1.73827131725846e8, 1.700705715168917e8, 1.69545952525199e8, 1.6872909289659202e8, 1.7354419431160915e8, 1.7114123096462288e8, 1.6859861132899177e8, 1.68779152871665e8, 1.6859928315793222e8, 1.7056306236811352e8, 1.6936079726167396e8, 1.7181743226963687e8, 1.8096476595600957e8, 1.7002401942994082e8, 1.7006125348473763e8, 1.6919057425576952e8, 1.746963559606877e8, 1.6910339107302448e8, 1.7430492724304593e8, 1.7088933480214176e8, 1.8793790053574124e8, 1.6869837889936787e8, 1.9186439818055052e8, 1.800820653831589e8, 1.7899772064557335e8, 1.6883153527518165e8, 1.6862085663535398e8, 1.7172896369843227e8, 1.6867359270494205e8, 1.6988842041005218e8, 1.7399807574132472e8, 1.6864595784092975e8, 1.693340883132707e8, 1.7248168114827034e8, 1.6864895416775948e8, 1.7460718764142606e8, 1.6982585352795798e8, 1.702596678548586e8, 1.720549638356921e8, 1.7938077851382422e8

In [16]:
println(k_best) #best split is 70% training and 30% validation
println(MSE_best)

6131
1.4664152769031666e7


In [17]:
X_train_wind = X_train_best
y_train_wind = y_train_best
X_valid_wind = X_valid_best
y_valid_wind = y_valid_best;

In [18]:
#transform back to dataframe to understand the coefficients
X_train_wind_df=DataFrame(X_train_wind, :auto)
X_valid_wind_df=DataFrame(X_valid_wind, :auto)
#rename columns with list of values
rename!(X_train_wind_df, names(df[:, 13:38]))
rename!(X_valid_wind_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_wind=DataFrame(Wind_Gen_MW = y_train_wind)
y_valid_wind=DataFrame(Wind_Gen_MW = y_valid_wind)
#save as CSV
CSV.write("X_train_wind.csv", X_train_wind_df)
CSV.write("y_train_wind.csv", y_train_wind)
CSV.write("X_valid_wind.csv", X_valid_wind_df)
CSV.write("y_valid_wind.csv", y_valid_wind);

In [19]:
stable_regression(X_train_best,y_train_best,k_best)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
[1.7246554114022803e8, 1.6871502251749748e8, 1.6969197254962716e8, 1.7113520212464988e8, 1.6937881014258614e8, 1.701854058122897e8, 1.6864304127982247e8, 1.6905143596079546e8, 1.740924324317909e8, 1.6939365100953174e8, 1.687643970111022e8, 1.686484410022318e8, 1.795903243529377e8, 1.7374443326172653e8, 1.704670812120694e8, 1.6861714346831545e8, 1.69937022935912e8, 1.6942298627692968e8, 1.7048624556184328e8, 1.695599702412038e8, 1.8102409607668522e8, 1.7193890633027732e8, 1.764555568851279e8, 1.7961553534891847e8, 1.714256632240873e8, 1.721546022955767e8, 1.6991973434195647e8, 1.686745291090197e8, 1.7259466787619472e8, 1.6915868281818497e8, 1.714294384242805e8, 1.770293715877902e8, 1.7052094794251573e8, 1.6879563892689925e8, 1.7707901679852775e8, 2.0139833167987597e8, 1.855023685853846e8, 1.7907239346496058e8, 1.8933050666009068e8, 1.689687415657922e8, 1.728327817830272e8, 1.7577480490151682e8, 1.

26-element Vector{Float64}:
   -347.71017131533137
      2.3017323857518464
    314.8927552777356
   -183.59875091854076
   2114.208483853177
   1513.8339794700864
   -204.32606785828122
   -217.89340252840702
    511.2800406198507
      5.760619759982874
   -106.28248151100182
    -15.024542909584014
    130.08221058685493
     -6.43872008183558
     91.2971259275709
  26009.9305105867
  25558.77966392912
  25943.37574930988
  69725.58831714292
  69267.85192890576
 -22369.6864610142
 -30187.653486430045
 100028.1264239812
 -94148.40245769739
   4929.4561240777
  -2469.8562371097782